In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('miss').getOrCreate()
file_path = '/FileStore/tables/ContainsNull_1_-020da.csv'

In [2]:
df = spark.read.csv(file_path, inferSchema=True,header=True)

In [3]:
df.show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [4]:
# Drop any row that contains missing data
#Drop the missing data
#You can use the .na functions for missing data. The drop command has the following parameters:
#
#df.na.drop(how='any', thresh=None, subset=None)
#
#* param how: 'any' or 'all'.
#
#   If 'any', drop a row if it contains any nulls.
#    If 'all', drop a row only if all its values are null.
#
#* param thresh: int, default None
#
#    If specified, drop rows that have less than `thresh` non-null values.
#    This overwrites the `how` parameter.
#
#* param subset: 
#    optional list of column names to consider.
df.na.drop().show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp4|Cindy|456.0|
+----+-----+-----+

In [5]:
# Has to have at least 2 NON-null values
df.na.drop(how='all', thresh=2).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [6]:
# Drop null for Named columns 
df.na.drop(subset=["Name"]).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp4|Cindy|456.0|
+----+-----+-----+

In [7]:
#how If 'any', drop a row if it contains any nulls.
df.na.drop(how='any').show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp4|Cindy|456.0|
+----+-----+-----+

In [8]:
# If 'all', drop a row only if all its values are null.
df.na.drop(how='all').show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [9]:
#Fill the missing values
#We can also fill the missing values with new values. If you have multiple nulls across multiple data types, Spark is actually smart enough to match up #the data types. For example:
df.na.fill('NEW VALUES').show()

+----+----------+-----+
 Id| Name|Sales|
+----+----------+-----+
emp1| John| null|
emp2|NEW VALUES| null|
emp3|NEW VALUES|345.0|
emp4| Cindy|456.0|
+----+----------+-----+

In [10]:
df.na.fill(0).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| 0.0|
emp2| null| 0.0|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [11]:
# Usually you should specify what columns you want to fill with the subset parameter
df.na.fill('No Names', subset=['Name']).show()

+----+--------+-----+
 Id| Name|Sales|
+----+--------+-----+
emp1| John| null|
emp2|No Names| null|
emp3|No Names|345.0|
emp4| Cindy|456.0|
+----+--------+-----+

In [12]:
help(df.na.fill)

Help on method fill in module pyspark.sql.dataframe:

fill(value, subset=None) method of pyspark.sql.dataframe.DataFrameNaFunctions instance
 Replace null values, alias for ``na.fill()``.
 :func:`DataFrame.fillna` and :func:`DataFrameNaFunctions.fill` are aliases of each other.
 
 :param value: int, long, float, string, bool or dict.
 Value to replace null values with.
 If the value is a dict, then `subset` is ignored and `value` must be a mapping
 from column name (string) to replacement value. The replacement value must be
 an int, long, float, boolean, or string.
 :param subset: optional list of column names to consider.
 Columns specified in subset that do not have matching data type are ignored.
 For example, if `value` is a string, and subset contains a non-string column,
 then the non-string column is simply ignored.
 
 >>> df4.na.fill(50).show()
 +---+------+-----+
age|height| name|
 +---+------+-----+
 10| 80|Alice|
 5| 50| Bob|
 50| 50| Tom|
 50| 50| null|
 +---+------+-----+
 
 >>> df5.na.fill(False).show()
 +----+-------+-----+
 age| name| spy|
 +----+-------+-----+
 10| Alice|false|
 5| Bob|false|
null|Mallory| true|
 +----+-------+-----+
 
 >>> df4.na.fill({'age': 50, 'name': 'unknown'}).show()
 +---+------+-------+
age|height| name|
 +---+------+-------+
 10| 80| Alice|
 5| null| Bob|
 50| null| Tom|
 50| null|unknown|
 +---+------+-------+
 
 .. versionadded:: 1.3.1

In [13]:
df.na.fill({"Name":"No Name","sales":0}).show()

+----+-------+-----+
 Id| Name|Sales|
+----+-------+-----+
emp1| John| 0.0|
emp2|No Name| 0.0|
emp3|No Name|345.0|
emp4| Cindy|456.0|
+----+-------+-----+

In [14]:
# A very common practice is to fill values with the mean value for the column, for example:
from pyspark.sql.functions import mean
mean_val = df.select(mean(df['Sales'])).collect()

In [15]:
mean_sales=mean_val[0][0]

In [16]:
df.na.fill(mean_sales,["Sales"]).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John|400.5|
emp2| null|400.5|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+